### Objective
The objective of this study is to develop a vision-based vehicle detection system for intelligent driving. *The paper proposes a vision-based vehicle detection system for intelligent driving. By using low-quality images from a monocular camera mounted on the front of a vehicle, the system aims to accurately detect and avoid obstacles (like other vehicles) to prevent accidents.*

### Methodology
The detection system is built around a CNN, which can efficiently extract features from input images and provide accurate recognitionof vehicles in real-time. The paper explores different CNN architectures and compares their performance, demonstrating that thisapproach yields high accuracy in vehicle detection under varying conditions.

### Key Components:
- CNN is utilized for feature extraction, classification, and vehicle detection from road images.
- The system involves a multi-layer CNN structure with convolution, pooling, and fully connected layers.
- The CNN-based system’s performance is compared to other models, such as Support Vector Machines (SVM) and Fully Connected Neural Networks (FFNN), with CNN outperforming them.

##### 1. Import Libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##### 2. Define the CNN Model


##### the CNN model has three convolutional layers followed by pooling layers and dense layers

In [ ]:
def create_cnn_model(input_shape=(64, 64, 3)):
    model = Sequential()
    
    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten and Fully Connected Layers
    model.add(Flatten())
    
    model.add(Dense(16, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    
    # Output Layer: 2 neurons (binary classification for "vehicle" or "no vehicle")
    model.add(Dense(2, activation='softmax'))
    
    return model

# Instantiate the model
model = create_cnn_model()
model.summary()

##### 3. Compile the Model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

##### 4. Prepare the Data

In [ ]:
# Define paths for train and validation directories
train_dir = '/path/to/train'  # Replace with your actual path
validation_dir = '/path/to/validation'  # Replace with your actual path

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Load Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

##### 5. Train the Model

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)

##### 6.  Evaluate the Model

In [ ]:
# Evaluate model performance on validation data
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

##### 7. Make Predictions

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_vehicle(model, img_path):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    if np.argmax(prediction) == 0:
        print("No Vehicle Detected")
    else:
        print("Vehicle Detected")

# Test on a new image
predict_vehicle(model, '/path/to/test_image.jpg')